In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Load the dataframe
df = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv', delimiter=',')
df.head()

In [ ]:
# Find percentage of null values
for item in df.columns:
    print(item, df[item].isna().mean()*100,'%')